In [1]:
%matplotlib inline

# import basic libraries
import yt
import numpy as np
import matplotlib.pyplot as plt

# import halo catalogue func
from yt.analysis_modules.halo_analysis.api import *

In [2]:
import tempfile
import shutil
import os

# Create temporary directory for storing files
tmpdir = tempfile.mkdtemp()

In [3]:
# load dataset
ds = yt.load('~/../../tigress/cen/LG4_2048_40pc/RD0090/redshift0090')

yt : [INFO     ] 2017-06-23 10:41:00,202 Parameters: current_time              = 207.43449741177
yt : [INFO     ] 2017-06-23 10:41:00,205 Parameters: domain_dimensions         = [128 128 128]
yt : [INFO     ] 2017-06-23 10:41:00,209 Parameters: domain_left_edge          = [ 0.  0.  0.]
yt : [INFO     ] 2017-06-23 10:41:00,212 Parameters: domain_right_edge         = [ 1.  1.  1.]
yt : [INFO     ] 2017-06-23 10:41:00,215 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2017-06-23 10:41:00,217 Parameters: current_redshift          = 3.9999999428215
yt : [INFO     ] 2017-06-23 10:41:00,219 Parameters: omega_lambda              = 0.712
yt : [INFO     ] 2017-06-23 10:41:00,222 Parameters: omega_matter              = 0.288
yt : [INFO     ] 2017-06-23 10:41:00,224 Parameters: hubble_constant           = 0.6933


In [10]:
ds.domain_dimensions

array([128, 128, 128])

In [4]:
ds.print_stats()

Parsing Hierarchy : 100%|██████████| 44224/44224 [00:04<00:00, 10271.84it/s]
yt : [INFO     ] 2017-06-23 10:41:11,456 Gathering a field list (this may take a moment.)


level	# grids	       # cells	     # cells^3
----------------------------------------------
  0	   256	       2097152	           128
  1	    60	        547304	            82
  2	   187	       2071552	           128
  3	   898	       9903616	           215
  4	  5193	      55377920	           382
  5	  9876	      53324192	           377
  6	  9632	      37417048	           335
  7	  7854	      22891480	           284
  8	  5388	      12002584	           229
  9	  1830	       6173384	           184
 10	  1321	       6643200	           188
 11	  1729	       9688816	           214
----------------------------------------------
   	 44224	     218138248


t = 2.07434497e+02 = 4.93026195e+16 s = 1.56230573e+09 years

Smallest Cell:
	Width: 3.301e-05 Mpc
	Width: 3.301e+01 pc
	Width: 6.810e+06 AU
	Width: 1.019e+20 cm


In [6]:
ds.field_list

[('all', 'creation_time'),
 ('all', 'dynamical_time'),
 ('all', 'initial_mass'),
 ('all', 'metallicity_fraction'),
 ('all', 'particle_index'),
 ('all', 'particle_mass'),
 ('all', 'particle_position_x'),
 ('all', 'particle_position_y'),
 ('all', 'particle_position_z'),
 ('all', 'particle_type'),
 ('all', 'particle_velocity_x'),
 ('all', 'particle_velocity_y'),
 ('all', 'particle_velocity_z'),
 ('all', 'typeia_fraction'),
 ('enzo', 'Dark_Matter_Density'),
 ('enzo', 'Density'),
 ('enzo', 'Electron_Density'),
 ('enzo', 'H2II_Density'),
 ('enzo', 'H2I_Density'),
 ('enzo', 'HII_Density'),
 ('enzo', 'HI_Density'),
 ('enzo', 'HM_Density'),
 ('enzo', 'HeIII_Density'),
 ('enzo', 'HeII_Density'),
 ('enzo', 'HeI_Density'),
 ('enzo', 'Metal_Density'),
 ('enzo', 'Temperature'),
 ('enzo', 'TotalEnergy'),
 ('enzo', 'x-velocity'),
 ('enzo', 'y-velocity'),
 ('enzo', 'z-velocity'),
 ('io', 'creation_time'),
 ('io', 'dynamical_time'),
 ('io', 'initial_mass'),
 ('io', 'metallicity_fraction'),
 ('io', 'part

In [5]:
# load halo dataset
halos_ds = yt.load('./rd0070/halo_catalogs/catalog/catalog0070.0.h5')

yt : [INFO     ] 2017-06-23 10:41:17,621 Parameters: current_time              = 3.75573046409e+16
yt : [INFO     ] 2017-06-23 10:41:17,622 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2017-06-23 10:41:17,625 Parameters: domain_left_edge          = [ 0.  0.  0.]
yt : [INFO     ] 2017-06-23 10:41:17,627 Parameters: domain_right_edge         = [  2.22535525e+25   2.22535525e+25   2.22535525e+25]
yt : [INFO     ] 2017-06-23 10:41:17,630 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2017-06-23 10:41:17,631 Parameters: current_redshift          = 4.99999998323
yt : [INFO     ] 2017-06-23 10:41:17,633 Parameters: omega_lambda              = 0.712
yt : [INFO     ] 2017-06-23 10:41:17,635 Parameters: omega_matter              = 0.288
yt : [INFO     ] 2017-06-23 10:41:17,636 Parameters: hubble_constant           = 0.6933


In [29]:
halos_ds.field_list

yt : [INFO     ] 2017-06-22 11:10:29,701 Allocating for 1.022e+04 particles (index particle type 'all')
yt : [INFO     ] 2017-06-22 11:10:29,719 Identified 3.545e+03 octs


[('all', 'particle_identifier'),
 ('all', 'particle_mass'),
 ('all', 'particle_position_x'),
 ('all', 'particle_position_y'),
 ('all', 'particle_position_z'),
 ('all', 'virial_radius'),
 ('halos', 'particle_identifier'),
 ('halos', 'particle_mass'),
 ('halos', 'particle_position_x'),
 ('halos', 'particle_position_y'),
 ('halos', 'particle_position_z'),
 ('halos', 'virial_radius')]

In [8]:
# Instantiate a catalog using those two paramter files
hc = HaloCatalog(data_ds=ds, halos_ds=halos_ds, output_dir=os.path.join(tmpdir, 'halo_catalog'))
hc.load()

yt : [INFO     ] 2017-06-23 11:19:19,885 Allocating for 9.178e+03 particles (index particle type 'all')
yt : [INFO     ] 2017-06-23 11:19:19,905 Identified 3.025e+03 octs


In [ ]:
# create projection plot
p = yt.ProjectionPlot(ds, "x", "Dark_Matter_Density", width=(1.5, 'Mpc'))
p.set_zlim(field="Dark_Matter_Density", zmin=1e23, zmax=1e27)
#p.annotate_halos(hc, factor = "particle_mass")
p.annotate_halos(hc)
p.display()